Jared Wright
jaredwright217@gmail.com

This file vectorizes the tweets and outputs the vectorized matrix to a csv


Mounting GoogleDrive data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Econ 484 Dream Team'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1othtLjYaMjzTu583kD2RAAz-OMGJGZuP/Econ 484 Dream Team


Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import regex as re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt

We create a DataFrame with the given data. After that, we create the 'populism' variable from the 'master variable'. After that we get count the values that say "populism" from the populism variable. 

In [ ]:
#import the raw data and select populism
tweets = pd.read_csv('/content/gdrive/My Drive/Econ 484 Dream Team/ml_twitter.csv')

tweets['populism'] = (tweets['master_frame'] == 'Populism')

print(tweets['populism'].value_counts())


% populist tweets: 0.1998
False    1422
True      355
Name: populism, dtype: int64


In [ ]:
#This code removes links from the tweets. It's a bit clunky but it works. My decision tree was using links to predict populism, 
#I guess there's a populist politician in Italy that likes to tweet links to his facebook profile...
def preprocess_text(text):
    text = text.lower()
    text = re.sub('\S+http?\S+', '', text)
    text = re.sub('\S+https?\S+', '', text)
    text = re.sub('\S+\.com\S+', '', text)
    text = re.sub('\S+\html\S+', '', text)
    text = re.sub('\S+goo\.gl?\S+', '', text)
    text = re.sub('http?\S+', '', text)
    text = re.sub('https?\S+', '', text)
    text = re.sub('\S+\.com', '', text)
    text = re.sub('\S+\html', '', text)
    text = re.sub('goo\.gl?\S+', '', text)
    text = re.sub(r'\d+', '', text)
    return text

#we make a list of the cleaned text and export it. 
clean_text_list = list()
for stringliteral in tweets['text_eng']:
  clean_text_list.append(preprocess_text(stringliteral))

clean_text = pd.DataFrame(clean_text_list, columns=['clean_text'])

clean_text.to_csv('/content/gdrive/My Drive/Econ 484 Dream Team/tweet_text.csv', index=False)
clean_text.head(5)

,clean_text
0,we are #envivo at a press conference. follow i...
1,"i deeply appreciate your valuable support, est..."
2,is it rational?
3,no to amnesty. criminals must go to jail # asi...
4,here is our government plan. i am fully convin...


In [ ]:
#vectorize the data
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(strip_accents='unicode', stop_words='english', min_df=10, preprocessor=preprocess_text)
tweet_features = vect.fit_transform(tweets['text_eng']).toarray()
feature_names = vect.get_feature_names()

print(feature_names)
print(tweet_features)
print(len(feature_names))

['able', 'access', 'accompanied', 'according', 'achieve', 'act', 'ago', 'agree', 'albert_rivera', 'ale_dibattista', 'allow', 'alternative', 'answer', 'anti', 'approved', 'april', 'ask', 'attack', 'aviva', 'away', 'based', 'beautiful', 'believe', 'berlusconi', 'best', 'better', 'big', 'bolsonaro', 'brazil', 'brazilian', 'bring', 'build', 'business', 'called', 'campaign', 'candidate', 'candidates', 'care', 'catalonia', 'center', 'change', 'child', 'children', 'choose', 'ciro', 'cirosim', 'citizen', 'citizens', 'citizenship', 'city', 'clear', 'close', 'closed', 'coalition', 'colombia', 'colombians', 'come', 'comes', 'commitment', 'commitments', 'committed', 'common', 'community', 'companies', 'congress', 'constitution', 'continue', 'corrupt', 'corruption', 'country', 'coup', 'course', 'crime', 'cs', 'culture', 'day', 'days', 'debate', 'debateine', 'debates', 'decent', 'defeat', 'defend', 'defense', 'del', 'democracy', 'democratic', 'development', 'di', 'dialogue', 'did', 'dignity', 'dirty

In [ ]:
#make a dataframe with the vectorized data
df = pd.DataFrame(tweet_features, columns=feature_names)
df

,able,access,accompanied,according,achieve,act,ago,agree,albert_rivera,ale_dibattista,allow,alternative,answer,anti,approved,april,ask,attack,aviva,away,based,beautiful,believe,berlusconi,best,better,big,bolsonaro,brazil,brazilian,bring,build,business,called,campaign,candidate,candidates,care,catalonia,center,...,turn,unemployment,united,uribe,use,useful,values,victims,video,violence,votapsoe,vote,voted,voters,votes,voting,vox,want,wanted,wants,war,watch,water,way,week,welcome,went,win,women,work,workers,working,world,year,years,yes,yesterday,young,youth,ªð
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1772,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1773,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1775,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
#drop if not alphabetical or is less than 3 characters. 
df_alpha = df
for string in df.columns:
  if (string.isalpha()==False):
    df_alpha.drop(columns=string, inplace=True)
  if (len(string) < 3):
    df_alpha.drop(columns=string, inplace=True)

df_alpha.head(5)

,able,access,accompanied,according,achieve,act,ago,agree,allow,alternative,answer,anti,approved,april,ask,attack,aviva,away,based,beautiful,believe,berlusconi,best,better,big,bolsonaro,brazil,brazilian,bring,build,business,called,campaign,candidate,candidates,care,catalonia,center,change,child,...,truth,turn,unemployment,united,uribe,use,useful,values,victims,video,violence,votapsoe,vote,voted,voters,votes,voting,vox,want,wanted,wants,war,watch,water,way,week,welcome,went,win,women,work,workers,working,world,year,years,yes,yesterday,young,youth
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#add populism to the dataframe
vectorized_data = pd.concat([df_alpha, tweets['populism']], axis=1)
vectorized_data

,able,access,accompanied,according,achieve,act,ago,agree,allow,alternative,answer,anti,approved,april,ask,attack,aviva,away,based,beautiful,believe,berlusconi,best,better,big,bolsonaro,brazil,brazilian,bring,build,business,called,campaign,candidate,candidates,care,catalonia,center,change,child,...,turn,unemployment,united,uribe,use,useful,values,victims,video,violence,votapsoe,vote,voted,voters,votes,voting,vox,want,wanted,wants,war,watch,water,way,week,welcome,went,win,women,work,workers,working,world,year,years,yes,yesterday,young,youth,populism
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1772,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
1773,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
1774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
1775,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,False


In [ ]:
#we export the data!
vectorized_data.to_csv('/content/gdrive/My Drive/Econ 484 Dream Team/tweet_features.csv', index=False)

In [ ]:
#this cell exports the model so that we can use it in other files
import pickle
from sklearn import datasets
%cd '/content/gdrive/My Drive/Econ 484 Dream Team'

with open('vectorizer.pkl', 'wb') as vectorizer_file:
  pickle.dump(vect, vectorizer_file)

/content/gdrive/.shortcut-targets-by-id/1othtLjYaMjzTu583kD2RAAz-OMGJGZuP/Econ 484 Dream Team
